# Data Preparation

In dit notebook zullen wij de features die wij in Data Exploration hebben gekozen, opschonen en zodanig aanpassen dat het door de modellen die wij interessant vinden, gebruikt kan worden.

In [2]:
%%capture
%run "main.ipynb"

In [3]:
from sklearn.preprocessing import TargetEncoder
import numpy as np
import json

# Data cleaning

Eerst halen we rijen met een meldnummer dat al eerder is voorgekomen uit de dataset. 
Uit onze bevindingen

Dit haalt zo'n 195000 entries weg en houden we er 350.000 over.

In [4]:
df = df[~df.index.duplicated(keep='first')]

### Missende waardes
De features die wij hebben gekozen, missen soms waardes in de kolommen.
Verschillende modellen kunnen hier niet mee omgaan. Om ervoor te zorgen dat deze onze modellen op de dataset kunnen werken, doen wij twee dingen. \
Wij verwijderen de rij(en) als missende waardes in deze kolom niet vaak voorkomen, en er geen goede waarde is om in te vullen. \
Ander proberen wij een waarde in te vullen. Dit kan een defaultwaarde zijn, die al in de dataset voorkomt, of wij kunnen zelf een nieuwe waarde hiervoor verzinnen.

Er zijn 1 rij die een prioriteit mist, deze rij halen we uit de database.

In [5]:
# NA's verwijderen omdat het er weinig zijn
df = df.dropna(subset=['stm_prioriteit'])

Er zijn 5 rijen die een geocode missen. Ook deze halen wij uit de database.

In [6]:
# stm_geo_mld
# Some computers differ in how they interpret a column without values
# So we remove both NaN values and empty strings
df = df[df.stm_geo_mld != '']
df = df.dropna(subset=['stm_geo_mld'])
df['stm_geo_mld'] = df['stm_geo_mld'].astype(float).astype(int).astype(str)

Voor missende oorzaakcodes, vullen wij de code 999 in. 
Deze wordt in de database al gebruikt, en is niet beschreven in de data dictionary die wij hebben gekregen.
Het is een default/missende waarde.

In [7]:
# stm_oorz_code
df['stm_oorz_code'] = df['stm_oorz_code'].fillna(221).astype(int).astype(str)

Voor missende techniekveld labels, vullen wij "X" in. Dit is de overige categorie, die al in de dataset terugkomt.

In [8]:
# stm_techn_mld
df['stm_techn_mld'] = df['stm_techn_mld'].replace('', 'X')
df['stm_techn_mld'] = df['stm_techn_mld'].fillna('X')

Voor het contractgebied vervangen wij missende waardes met 999. Deze komt in de dataset niet voor en is vergeleken met de andere waardes in deze kolom, overduidelijk een default/overig value.

In [9]:
# stm_contractgeb_mld
df['stm_contractgeb_mld'] = df['stm_contractgeb_mld'].fillna(999)
df['stm_contractgeb_mld'] = df['stm_contractgeb_mld'].astype(float).astype(int).astype(str)

Voor de oorzaakgroep vullen wij voor missende waardes "ONBK" in. Dit staat voor onbekend.

In [11]:
# stm_oorz_groep
df['stm_oorz_groep'] = df['stm_oorz_groep'].replace('', 'ONBK', regex=True)

# Data preparation

Later gebruiken wij target-encoding, en hiervoor moeten wij een train/test split maken. \
Anders zouden de waardes, waarop wij het model testen, deels het model al hebben beïnvloed, en dat schaadt de testvaliditeit.

In [12]:
# Voor random state 33 zijn alle unieke waardes waarvoor wij target encoding gebruiken gerepresenteerd.
# df_train is 80% van de dataset
df_train = df.sample(frac = 0.8, random_state=33, weights=df['anm_tot_fh'])
# df_test is de overige 20%
df_test = df.drop(df_train.index)

## Dummy encoding

Dummy encoding is een makkelijke en simpele manier om features met een lage kardinaliteit te encoderen.

### stm_prioriteit

Deze variabele gebruiken wij als een ordinale meetwaarde en hoeft dus niet veranderd te worden.

In [ ]:
# In het geval dummies gewenst zijn, kan je de volgende twee regels, en enkele regels verderop in de code uncommenten
# prioriteit_dummies_train = pd.get_dummies(df_train['stm_prioriteit'], prefix='prio')
# prioriteit_dummies_test = pd.get_dummies(df_test['stm_prioriteit'], prefix='prio')

### stm_techn_mld

In [13]:
techn_veld_dummies_train = pd.get_dummies(df_train['stm_techn_mld'], prefix='techn_veld')
techn_veld_dummies_test = pd.get_dummies(df_test['stm_techn_mld'], prefix='techn_veld')

### stm_oorz_groep

In [14]:
oorz_groep_dummies_train = pd.get_dummies(df_train['stm_oorz_groep'], prefix='oorzgr')
oorz_groep_dummies_test = pd.get_dummies(df_test['stm_oorz_groep'], prefix='oorzgr')

## Target encoding

Target encoding geeft de nominale features een score op basis van het effect
wat zij hebben op de target variabele.
Het zet een nominale waarde in zekere mate om tot een continue waarde.
Dit zorgt ervoor dat bepaalde modellen de feature kunnen gebruiken die dit anders niet konden,
en geeft bepaalde modellen, zoals de decisionTree, een manier om de dataset op een grootschaligere manier
te splitsen. Dus in plaats van dat het splitst op, de waarde is gelijk aan 1 van de 200 nominale waardes,
splitst het op een waarde die representeert welk effect de waarde heeft op de target variabele.
Waardes die over het algemeen in een lagere target variabele waarde resulteren, splitsen van de rest.

In [15]:
# Setup TargetEncoder
# Onze target is een duur in minuten, dus continue
tEnc = TargetEncoder(target_type="continuous", random_state=42)
y = df_train['anm_tot_fh']

### stm_oorz_code

In [16]:
# Fit het model op een feature in de traindata en maak een kolom aan met de geëncodeerde waardes
X = np.array(df_train['stm_oorz_code']).reshape(-1, 1)
tEnc.fit(X,y)
df_train['oorz_code_enc'] = tEnc.transform(X)

In [17]:
# Een dictionary met de encodings (voor het dashboard)
oorzc_dict = {}
for i in range(len(tEnc.categories_[0])):
    cat, enc = tEnc.categories_[0][i], tEnc.encodings_[0][i]
    oorzc_dict[cat] = enc
# oorzc_dict

In [18]:
# In de testdata dezelfde kolom aanmaken met de encodings van de traindata
df_test['oorz_code_enc'] = df_test['stm_oorz_code'].apply(lambda x : oorzc_dict[x])

### stm_geo_mld

In [19]:
# Fit het model op een feature in de traindata en maak een kolom aan met de geëncodeerde waardes
X = np.array(df_train['stm_geo_mld']).reshape(-1, 1)
tEnc.fit(X,y)
df_train['geo_code_enc'] = tEnc.transform(X)

In [20]:
# Een dictionary met de encodings (voor het dashboard)
geo_dict = {}
for i in range(len(tEnc.categories_[0])):
    cat, enc = tEnc.categories_[0][i], tEnc.encodings_[0][i]
    geo_dict[cat] = enc
# geo_dict

In [21]:
# In de testdata dezelfde kolom aanmaken met de encodings van de traindata
df_test['geo_code_enc'] = df_test['stm_geo_mld'].apply(lambda x : geo_dict[x])

### stm_contractgeb_mld

In [22]:
# Fit het model op een feature in de traindata en maak een kolom aan met de geëncodeerde waardes
X = np.array(df_train['stm_contractgeb_mld']).reshape(-1, 1)
tEnc.fit(X,y)
df_train['contractgb_enc'] = tEnc.transform(X)

In [23]:
# Een dictionary met de encodings (voor het dashboard)
contrgb_dict = {}
for i in range(len(tEnc.categories_[0])):
    cat, enc = tEnc.categories_[0][i], tEnc.encodings_[0][i]
    contrgb_dict[cat] = enc
# contrgb_dict

In [24]:
# In de testdata dezelfde kolom aanmaken met de encodings van de traindata
df_test['contractgb_enc'] = df_test['stm_contractgeb_mld'].apply(lambda x : contrgb_dict[x])

### stm_techn_mld

In [25]:
# Fit het model op een feature in de traindata en maak een kolom aan met de geëncodeerde waardes
X = np.array(df_train['stm_techn_mld']).reshape(-1, 1)
tEnc.fit(X,y)
df_train['techn_enc'] = tEnc.transform(X)

In [26]:
# Een dictionary met de encodings (voor het dashboard)
techn_dict = {}
for i in range(len(tEnc.categories_[0])):
    cat, enc = tEnc.categories_[0][i], tEnc.encodings_[0][i]
    techn_dict[cat] = enc
# techn_dict

In [27]:
# In de testdata dezelfde kolom aanmaken met de encodings van de traindata
df_test['techn_enc'] = df_test['stm_techn_mld'].apply(lambda x : techn_dict[x])

In [ ]:
df_test.sample(20)

# Geprepareerde data exporteren

Wij exporteren de dataframes naar csv bestanden, zodat de modellen in andere notebooks het makkelijk, snel en geprepareerd in kunnen laden.

### Train dataframa

In [28]:
df_train = df_train.reset_index(drop=True)
# prioriteit_dummies_train = prioriteit_dummies.reset_index(drop=True)
prioriteit = df_train['stm_prioriteit'].reset_index(drop=True)
oorz_code_enc = df_train['oorz_code_enc'].reset_index(drop=True)
geo_code_enc = df_train['geo_code_enc'].reset_index(drop=True)
contractgb_enc = df_train['contractgb_enc'].reset_index(drop=True)
techn_enc = df_train['techn_enc'].reset_index(drop=True)
techn_veld_dummies = techn_veld_dummies_train.reset_index(drop=True)
oorz_groep_dummies = oorz_groep_dummies_train.reset_index(drop=True)
# fh_prog = df['stm_progfh_in_duur']

features_to_use = [
    # prioriteit_dummies
    prioriteit,
    oorz_code_enc,
    geo_code_enc,
    contractgb_enc,
    techn_enc,
    techn_veld_dummies,
    oorz_groep_dummies
]

train_df = pd.concat([df_train['anm_tot_fh'], *features_to_use], axis=1)

In [29]:
train_df.to_pickle("data/train_df.pkl")

### Test dataframe

In [30]:
df_test = df_test.reset_index(drop=True)
# prioriteit_dummies_test = prioriteit_dummies.reset_index(drop=True)
prioriteit = df_test['stm_prioriteit'].reset_index(drop=True)
oorz_code_enc = df_test['oorz_code_enc'].reset_index(drop=True)
geo_code_enc = df_test['geo_code_enc'].reset_index(drop=True)
contractgb_enc = df_test['contractgb_enc'].reset_index(drop=True)
techn_enc = df_test['techn_enc'].reset_index(drop=True)
techn_veld_dummies = techn_veld_dummies_test.reset_index(drop=True)
oorz_groep_dummies = oorz_groep_dummies_test.reset_index(drop=True)
# fh_prog = df['stm_progfh_in_duur']

features_to_use = [
    # prioriteit_dummies
    prioriteit,
    oorz_code_enc,
    geo_code_enc,
    contractgb_enc,
    techn_enc,
    # techn_veld_dummies,
    # oorz_groep_dummies
]

test_df = pd.concat([df_test['anm_tot_fh'], *features_to_use], axis=1)

In [31]:
test_df.to_pickle("data/test_df.pkl")

## Json files

Hier maken wij de json-files met de encoded values van de unieke waarden in de kolom, zodat dit beschikbaar is in het dashboard.

In [32]:
# Dictionary van de dictionaries
encodings = {'oorzaak_code': oorzc_dict,
 'geo_code': geo_dict,
 'contractgb': contrgb_dict,
 'techn_veld': techn_dict
}
with open('data/feature_encodings.json', 'w') as outfile:
    json.dump(encodings, outfile)